In [1]:
from bs4 import BeautifulSoup
import numpy as np
import requests

In [2]:
# Build the links for crawling
home  = "http://www.ipaidabribe.com/reports/all#gsc.tab=0"
left  = "http://www.ipaidabribe.com/reports/all?page="
right = "#gsc.tab=0"
mid   = np.arange(10, 1000, 10)
links = [home]
for x in mid:
    temp = str(left) + str(x) + str(right)
    links.append(temp)

In [ ]:
# Get all the pages for extraction
pages = []
for x in links:
    print("Processing page...")
    html = requests.get(str(x)).text
    soup = BeautifulSoup(html, 'html5lib')
    pages.append(soup)

Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...
Processing page...


In [97]:
# Extracting the titles from the bribes taken.
# This is only for a single website.
title      = []
date       = []
bribe      = []
department = []
views      = []
city       = []

for soup in pages:
    # Title of the offense.
    title_raw = soup.find_all("h3")
    for x in title_raw:
        temp = x.text
        temp = temp.split()
        temp = " ".join(temp)
        title.append(temp)
    
    # Date of the offense.
    date_raw   = soup.find_all("span", class_="date") 
    for x in date_raw:
        date.append(x.text)

    # Amount of bribe.
    bribe_raw = soup.find_all("li", class_="paid-amount")
    for x in bribe_raw:
        temp = x.text
        temp = temp.split()
        temp = " ".join(temp)
        temp = temp.split("INR")[-1]
        temp = temp.replace(",", "")
        bribe.append(int(temp))
    
    # Name of department
    department_raw = soup.find_all("li", class_="transaction")
    for x in department_raw:
        temp = x.text
        temp = temp.split()
        temp = " ".join(temp)
        department.append(temp)
    
    # Number of views
    views_raw = soup.find_all("li", class_="views")
    for x in views_raw:
        temp = x.text
        temp = temp.replace(" views", "")
        views.append(int(temp))
    
    # City
    city_raw = soup.find_all("a", class_="location")
    for x in city_raw:
        temp = x.text
        temp = temp.split()
        temp = " ".join(temp)
        temp = temp.replace(" ,", ",")
        city.append(temp)
    

[]
